In [1]:
import numpy as np 
import pandas as pd


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
            print(os.path.join(dirname, filename))



/kaggle/input/samhitika/SanskritDataset/test.sn
/kaggle/input/samhitika/SanskritDataset/train.sn
/kaggle/input/samhitika/SanskritDataset/dev.sn


In [2]:
from datasets import load_dataset

ds = load_dataset("rahular/itihasa")

README.md:   0%|          | 0.00/3.03k [00:00<?, ?B/s]

itihasa.py:   0%|          | 0.00/4.89k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/16.4M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/2.61M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/75162 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6149 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11722 [00:00<?, ? examples/s]

In [3]:
print(ds.keys())  
train_ds      = ds["train"]
validation_ds = ds["validation"]
test_ds       = ds["test"]


dict_keys(['train', 'validation', 'test'])


In [4]:
from collections import Counter

sanskrit_shlokas = [shloka["translation"]["sn"] for shloka in train_ds] 



print(train_ds[100])
print(sanskrit_shlokas[1000])

# for shloka in sanskrit_shlokas:
#     print(Counter(shloka))



{'translation': {'en': 'Then having duly performed his ablutions at that holy spot, the reverend sage retraced his steps, pondering over the incidents touching the sloka.', 'sn': 'सोऽभिषेकं ततः कृत्वा तीर्थे तस्मिन् यथाविधि। तमेव चिन्तयत्रर्थमुपावर्तत वै मुनिः॥'}}
सिद्धाश्रमनिवासं च राक्षसानां वधं तथा। तत्रागमनमव्यग्रं विशालायाश्च दर्शनम्॥ अहल्यादर्शनं चैव गौतमेन समागमम्। महाधनुषि जिज्ञासां कर्तुमागमनं तथा॥


In [5]:
import re 

combined_shlokas= "".join(sanskrit_shlokas) 

cleaned_shlokas = re.sub(r'[|।॥]+', '', combined_shlokas)


trigrams = [cleaned_shlokas[i:i+3] for i in range(len(cleaned_shlokas)-2)]
trigram_freq = Counter(trigrams) 

# print(trigram_freq) 




In [6]:
unique_chars = set(cleaned_shlokas) 
unique_chars.discard(" ")

vocab_trigram = [trigram for trigram, ct in trigram_freq.items() if ct>20]
init_vocab = list(unique_chars) + vocab_trigram

init_vocab = sorted(init_vocab) 

print(f"chars ct: {len(unique_chars)},  frequent trigrams ct: {len(vocab_trigram)}")
print("Example entries:", init_vocab[:50])



chars ct: 133,  frequent trigrams ct: 13488
Example entries: ['  अ', '  क', '  त', '  द', '  न', '  प', '  म', '  य', '  र', '  व', '  श', '  स', ' : ', ' अं', ' अक', ' अग', ' अङ', ' अच', ' अज', ' अत', ' अथ', ' अद', ' अध', ' अन', ' अप', ' अब', ' अभ', ' अम', ' अय', ' अर', ' अल', ' अव', ' अश', ' अष', ' अस', ' अह', ' आक', ' आख', ' आग', ' आच', ' आज', ' आत', ' आद', ' आध', ' आन', ' आप', ' आभ', ' आम', ' आय', ' आर']


In [7]:
special_tokens = [ "<UNK>", "<BOS>", "<EOS>"]
final_vocab = init_vocab + special_tokens 

print(len(final_vocab))

13624


In [8]:
from datasets import load_dataset
from collections import Counter, defaultdict
import re
import math
from typing import List, Dict, Tuple

class SanskritUnigramTokenizer:
    def __init__(self, target_vocab_size: int = 8000):
        self.vocab = []
        self.token_probs = {}
        self.target_vocab_size = target_vocab_size
        self.special_tokens = ["<UNK>", "<BOS>", "<EOS>"]
        
    def get_best_segmentation_viterbi(self, text: str) -> List[str]:
        """Use Viterbi algorithm to find best segmentation efficiently"""
        if not text:
            return []
            
        n = len(text)
        # dp[i] = (best_log_prob, best_previous_position)
        dp = [(-float('inf'), -1) for _ in range(n + 1)]
        dp[0] = (0.0, -1)  
        
        for i in range(n + 1):
            if dp[i][0] == -float('inf'):
                continue
                
            for token in self.vocab:
                if i + len(token) <= n and text[i:i+len(token)] == token:
                    new_pos = i + len(token)
                    token_prob = math.log(self.token_probs.get(token, 1e-10))
                    new_prob = dp[i][0] + token_prob
                    
                    if new_prob > dp[new_pos][0]:
                        dp[new_pos] = (new_prob, i)
        
        # Backward pass to reconstruct best path
        if dp[n][0] == -float('inf'):
            # char level tokenization 
            return [char if char in self.vocab else "<UNK>" for char in text]
        
        # Reconstruct path
        result = []
        pos = n
        while pos > 0:
            prev_pos = dp[pos][1]
            token = text[prev_pos:pos]
            result.append(token)
            pos = prev_pos
        
        return result[::-1]  
    
    def update_token_counts(self, corpus: List[str]) -> Dict[str, int]:
        """Count token usage in best segmentations across corpus"""
        token_counts = defaultdict(int)
        
        print("Segmenting corpus and counting tokens...")
        for i, text in enumerate(corpus):
            if i % 100 == 0:  
                print(f"Processed {i}/{len(corpus)} texts")
            
            # Clean text
            clean_text = re.sub(r'[|।॥\s]+', '', text).strip()
            if not clean_text:
                continue
                
            # Viterbi implementation 
            best_seg = self.get_best_segmentation_viterbi(clean_text)
            
            # Count tokens in this segmentation
            for token in best_seg:
                token_counts[token] += 1
        
        return dict(token_counts)
    
    def update_probabilities(self, token_counts: Dict[str, int]):
        """Update token probabilities based on usage counts"""
        total_tokens = sum(token_counts.values())
        
        if total_tokens == 0:
            prob = 1.0 / len(self.vocab)
            self.token_probs = {token: prob for token in self.vocab}
            return
        
        # Calculate probabilities with smoothing
        
        self.token_probs = {}
        for token in self.vocab:
            count = token_counts.get(token, 0)
            # add-1 smoothing 
            self.token_probs[token] = (count + 1) / (total_tokens + len(self.vocab))
    
    def prune_vocabulary(self, token_counts: Dict[str, int], min_count: int = 2):
        """remove rarely used tokens from vocabulary"""
        # special tokens and frequently used tokens 
        new_vocab = []
        
        for token in self.vocab:
            if token in self.special_tokens:
                new_vocab.append(token)  # Always keep special tokens
            elif token_counts.get(token, 0) >= min_count:
                new_vocab.append(token)
        
        # keep top tokens by frequency if 
        if len(new_vocab) > self.target_vocab_size:
            # Sort by count (excluding special tokens)
            regular_tokens = [t for t in new_vocab if t not in self.special_tokens]
            regular_tokens.sort(key=lambda t: token_counts.get(t, 0), reverse=True)
            
            # Keep top tokens + special tokens
            keep_count = self.target_vocab_size - len(self.special_tokens)
            new_vocab = self.special_tokens + regular_tokens[:keep_count]
        
        print(f"Pruned vocabulary: {len(self.vocab)} → {len(new_vocab)} tokens")
        self.vocab = new_vocab
    
    def train(self, corpus: List[str], max_iterations: int = 5):  # Reduced iterations
        """Train the unigram tokenizer"""
        print(f"Starting training with {len(self.vocab)} initial vocabulary size")
        
        # Initialize probabilities uniformly
        initial_prob = 1.0 / len(self.vocab)
        self.token_probs = {token: initial_prob for token in self.vocab}
        
        for iteration in range(max_iterations):
            print(f"\nIteration {iteration + 1}/{max_iterations}")
            print(f"Current vocabulary size: {len(self.vocab)}")
            
            # Count token usage in best segmentations
            token_counts = self.update_token_counts(corpus)
            
            # Update probabilities
            self.update_probabilities(token_counts)
            
            if iteration < max_iterations - 1:
                old_size = len(self.vocab)
                self.prune_vocabulary(token_counts, min_count=max(1, iteration))
                
                if abs(old_size - len(self.vocab)) < 10:
                    print("Vocabulary stabilized, stopping early")
                    break
        
        print(f"\nTraining complete! Final vocabulary size: {len(self.vocab)}")
    
    def tokenize(self, text: str) -> List[str]:
        """Tokenize text using trained model"""
        clean_text = re.sub(r'[|।॥\s]+', '', text).strip()
        if not clean_text:
            return []
        return self.get_best_segmentation_viterbi(clean_text)
    
    def save_vocab(self, filepath: str):
        """Save vocabulary to file"""
        with open(filepath, 'w', encoding='utf-8') as f:
            for token in self.vocab:
                prob = self.token_probs.get(token, 0)
                f.write(f"{token}\t{prob}\n")
        print(f"Vocabulary saved to {filepath}")

print("Loading dataset...")
ds = load_dataset("rahular/itihasa")
train_ds = ds["train"]
test_ds = ds["test"]  

print("Extracting Sanskrit shlokas...")
sanskrit_shlokas = [shloka["translation"]["sn"] for shloka in train_ds]

print("Building initial vocabulary...")

# smaller sample of vocab building 

sample_shlokas = sanskrit_shlokas[:10000] 
combined_shlokas = "".join(sample_shlokas)
cleaned_shlokas = re.sub(r'[|।॥\s]+', '', combined_shlokas)

unique_chars = list(set(cleaned_shlokas))

# frequent bigrams and trigrams 
bigrams = [cleaned_shlokas[i:i+2] for i in range(len(cleaned_shlokas)-1)]
trigrams = [cleaned_shlokas[i:i+3] for i in range(len(cleaned_shlokas)-2)]

bigram_freq = Counter(bigrams)
trigram_freq = Counter(trigrams)
vocab_bigram = [bg for bg, ct in bigram_freq.items() if ct > 100]
vocab_trigram = [tg for tg, ct in trigram_freq.items() if ct > 50]

# Build smaller initial vocabulary
init_vocab = unique_chars + vocab_bigram + vocab_trigram
init_vocab = sorted(list(set(init_vocab)))  

print(f"Initial vocabulary: {len(unique_chars)} chars + {len(vocab_bigram)} bigrams + {len(vocab_trigram)} trigrams = {len(init_vocab)} total")

# Initialize and train tokenizer with smaller vocab
tokenizer = SanskritUnigramTokenizer(target_vocab_size=3000)  
tokenizer.vocab = init_vocab + tokenizer.special_tokens

print("Starting tokenizer training...")

train_subset = sanskrit_shlokas
tokenizer.train(train_subset, max_iterations=3)

print("\n" + "="*50)
print("TESTING TOKENIZER")
print("="*50)

test_texts = [test_ds[i]["translation"]["sn"] for i in range(5)]

for text in test_texts:
    tokens = tokenizer.tokenize(text)
    print(f"Text: {text}")
    print(f"Tokens: {tokens}")
    print(f"Token count: {len(tokens)}")
    print()

# save trained vocabularly
tokenizer.save_vocab("sanskrit_vocab.txt")

print(f"Final vocabulary size: {len(tokenizer.vocab)}")
print("Top 20 most probable tokens:")
sorted_tokens = sorted(tokenizer.token_probs.items(), key=lambda x: x[1], reverse=True)
for token, prob in sorted_tokens[:20]:
    print(f"{token}: {prob:.6f}")

Loading dataset...
Extracting Sanskrit shlokas...
Building initial vocabulary...
Initial vocabulary: 102 chars + 806 bigrams + 3222 trigrams = 4130 total
Starting tokenizer training...
Starting training with 4133 initial vocabulary size

Iteration 1/3
Current vocabulary size: 4133
Segmenting corpus and counting tokens...
Processed 0/75162 texts
Processed 100/75162 texts
Processed 200/75162 texts
Processed 300/75162 texts
Processed 400/75162 texts
Processed 500/75162 texts
Processed 600/75162 texts
Processed 700/75162 texts
Processed 800/75162 texts
Processed 900/75162 texts
Processed 1000/75162 texts
Processed 1100/75162 texts
Processed 1200/75162 texts
Processed 1300/75162 texts
Processed 1400/75162 texts
Processed 1500/75162 texts
Processed 1600/75162 texts
Processed 1700/75162 texts
Processed 1800/75162 texts
Processed 1900/75162 texts
Processed 2000/75162 texts
Processed 2100/75162 texts
Processed 2200/75162 texts
Processed 2300/75162 texts
Processed 2400/75162 texts
Processed 2500

In [9]:
test_texts  = [shloka["translation"]["sn"] for shloka in test_ds] 
print(test_texts[1000])

 


धनत्यागः सुखत्यागो देशत्यागोऽपि वाऽनघ। वयस्यार्थे प्रवर्तन्ते स्नेहं दृष्ट्वा तथाविधम्॥
